In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime

In [2]:
nypd = pd.read_csv(r'C:\Users\natasha\Downloads\NYPD_Arrest_Data__Year_to_Date_.csv')

In [22]:
nypd.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
0,206892169,2019-12-31,907.0,"IMPAIRED DRIVING,DRUG",347.0,INTOXICATED & IMPAIRED DRIVING,VTL11920U4,M,M,33,0,25-44,M,WHITE,1000916,245710,40.841085,-73.939769
1,206888084,2019-12-31,739.0,"FRAUD,UNCLASSIFIED-FELONY",112.0,THEFT-FRAUD,PL 1908301,F,Q,101,0,25-44,M,BLACK HISPANIC,1055868,156522,40.596012,-73.742116
2,206890433,2019-12-31,122.0,"HOMICIDE, NEGLIGENT, VEHICLE,",102.0,HOMICIDE-NEGLIGENT-VEHICLE,PL 1251201,F,M,23,0,18-24,M,BLACK,997462,227024,40.789803,-73.952289
3,206890441,2019-12-31,268.0,CRIMINAL MIS 2 & 3,121.0,CRIMINAL MISCHIEF & RELATED OF,PL 1450502,F,S,120,0,18-24,M,BLACK,962822,174282,40.645023,-74.077217
4,206890973,2019-12-31,101.0,ASSAULT 3,344.0,ASSAULT 3 & RELATED OFFENSES,PL 1200001,M,M,25,97,25-44,M,BLACK,1004138,226326,40.787875,-73.928182


In [4]:
nypd.shape

(214617, 18)

In [5]:
#nypd = nypd.drop(columns = [])

In [6]:
#nypd.head()

In [7]:
#nypd..shape

In [8]:
nypd.isnull().values.any()

True

In [9]:
nypd.isnull().sum().sum()

1887

<font color ='purple'> After examining the NYPD dataset, we observed that there are 1887 missing values. </font>


In [10]:
nypd = nypd.dropna()

<font color='purple'>We dropped all records with missing data, since filling it with zeros or the mean would not be the best choice, since no calculations will be used to analyse the data. Those numeric values are data that reflects real facts/numbers and should not be manipulated else it can result in misleading analysis. We only lost less than 1% of our data by dropping them. </font>


In [11]:
nypd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213089 entries, 0 to 214616
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   ARREST_KEY         213089 non-null  int64  
 1   ARREST_DATE        213089 non-null  object 
 2   PD_CD              213089 non-null  float64
 3   PD_DESC            213089 non-null  object 
 4   KY_CD              213089 non-null  float64
 5   OFNS_DESC          213089 non-null  object 
 6   LAW_CODE           213089 non-null  object 
 7   LAW_CAT_CD         213089 non-null  object 
 8   ARREST_BORO        213089 non-null  object 
 9   ARREST_PRECINCT    213089 non-null  int64  
 10  JURISDICTION_CODE  213089 non-null  int64  
 11  AGE_GROUP          213089 non-null  object 
 12  PERP_SEX           213089 non-null  object 
 13  PERP_RACE          213089 non-null  object 
 14  X_COORD_CD         213089 non-null  int64  
 15  Y_COORD_CD         213089 non-null  int64  
 16  La

In [20]:
nypd['ARREST_DATE'] = pd.to_datetime(nypd['ARREST_DATE'], format = "%m/%d/%Y")


In [21]:
nypd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 213089 entries, 0 to 214616
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ARREST_KEY         213089 non-null  int64         
 1   ARREST_DATE        213089 non-null  datetime64[ns]
 2   PD_CD              213089 non-null  float64       
 3   PD_DESC            213089 non-null  object        
 4   KY_CD              213089 non-null  float64       
 5   OFNS_DESC          213089 non-null  object        
 6   LAW_CODE           213089 non-null  object        
 7   LAW_CAT_CD         213089 non-null  object        
 8   ARREST_BORO        213089 non-null  object        
 9   ARREST_PRECINCT    213089 non-null  int64         
 10  JURISDICTION_CODE  213089 non-null  int64         
 11  AGE_GROUP          213089 non-null  object        
 12  PERP_SEX           213089 non-null  object        
 13  PERP_RACE          213089 non-null  object  

In [23]:
counts = nypd.OFNS_DESC.value_counts()
percent = nypd.OFNS_DESC.value_counts(normalize = True).mul(100).round(2).astype(str) + '%'
pd.DataFrame({'counts': counts, 'percent': percent})

,counts,percent
ASSAULT 3 & RELATED OFFENSES,31988,15.01%
PETIT LARCENY,21627,10.15%
DANGEROUS DRUGS,21071,9.89%
FELONY ASSAULT,15208,7.14%
VEHICLE AND TRAFFIC LAWS,14243,6.68%
...,...,...
HOMICIDE-NEGLIGENT-VEHICLE,5,0.0%
PARKING OFFENSES,3,0.0%
LOITERING FOR DRUG PURPOSES,2,0.0%
NYS LAWS-UNCLASSIFIED VIOLATION,2,0.0%


##
<font color = 'purple' > In 2019, the highest Offence was 'Assault related Offensed which made up 15%, followed by Petit Larceny with 10%.<\font>

In [64]:
Arrest_by_boro = nypd.groupby('ARREST_BORO').OFNS_DESC.count()


Arrest_by_boro = pd.DataFrame(Arrest_by_boro.sort_values(ascending = False))

Arrest_by_boro


,OFNS_DESC
ARREST_BORO,
K,58077
M,53172
B,48516
Q,44329
S,8995


<font color = 'purple'> K(Brooklyn) had 58,077 Offences, the highest number of Offences for the year 2019 as shown above, followed by Manhattan with 53,172, then the Bronx, Queens and Staten island, with 48,516, 44,329 and 8,995 respectively.<font/>